<a href="https://colab.research.google.com/github/AtiqSha/colorised_b-w_image/blob/master/Colorized_BW_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#####Mount google drive storage

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#####Import packages and set image path

In [ ]:
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf
import cv2

In [ ]:
INPUT_PATH = '/content/drive/MyDrive/bw/inputs/'
OUTPUT_PATH = '/content/drive/MyDrive/bw/outputs/'
TRAIN_IMAGE = '{}/train.jpg'.format(INPUT_PATH)
TEST_COLOR_IMAGE = '{}/test_colorised.jpg'.format(INPUT_PATH)

#####Prepare training image - test on single image due to lack of storage

In [ ]:
# Get images
train_image = img_to_array(load_img(TRAIN_IMAGE))
train_image = np.array(train_image, dtype=float)
height, width, channel = train_image.shape
print(train_image.shape)

In [ ]:
X = rgb2lab(1.0/255*train_image)[:,:,0]
Y = rgb2lab(1.0/255*train_image)[:,:,1:]
Y /= 128
X = X.reshape(1, height, width, 1)
Y = Y.reshape(1, height, width, 2)

#####Build neural network

In [ ]:
model = Sequential()
model.add(InputLayer(input_shape=(None, None, 1)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='rmsprop',loss='mse')

In [ ]:
model.fit(x=X, y=Y, batch_size=1, epochs=100)

In [ ]:
print(model.evaluate(X, Y, batch_size=1))
output = model.predict(X)
output *= 128
# Output colorizations
cur = np.zeros((height, width, 3))
cur[:,:,0] = X[0][:,:,0]
cur[:,:,1:] = output[0]

In [ ]:
imsave('{}/img_results.png'.format(OUTPUT_PATH), lab2rgb(cur))

In [ ]:
# save model in h5 ext
MODEL_PATH = '{}/colorised_test_model.h5'.format(OUTPUT_PATH)
model.save(MODEL_PATH)

#####Test different image

In [ ]:
## reload trained model if necessary
MODEL_PATH = '{}/colorised_test_model.h5'.format(OUTPUT_PATH)'
model = tf.keras.models.load_model(MODEL_PATH)

In [ ]:
#test colorised image to compare
test_image = img_to_array(load_img(TEST_COLOR_IMAGE))
test_image = np.array(test_image, dtype=float)
height2, width2, channel2 = test_coloured_image.shape
print(test_coloured_image.shape)

Xtest = rgb2lab(1.0/255*test_coloured_image)[:,:,0]
Ytest = rgb2lab(1.0/255*test_coloured_image)[:,:,1:]
Ytest /= 128
Xtest = Xtest.reshape(1, height2, width2, 1)
Ytest = Ytest.reshape(1, height2, width2, 2)
output_test = model.predict(Xtest)
output_test *= 128

# Output colorizations
cur_test = np.zeros((height2, width2, 3))
cur_test[:,:,0] = Xtest[0][:,:,0]
cur_test[:,:,1:] = output_test[0]
imsave('{}/img_test_results.png'.format(OUTPUT_PATH), lab2rgb(cur_test))